In [1]:
!pip install music21 tensorflow numpy pandas

In [2]:
# STEP 2: Load MIDI Data (Bach Chorales from music21)
from music21 import corpus, converter, instrument, note, chord

notes = []

# Loop through a few Bach chorales
for i, chorale in enumerate(corpus.chorales.Iterator()[:20]):  # 20 short songs
    # s = chorale.stream # This line caused the error
    parts = instrument.partitionByInstrument(chorale) # Use chorale directly
    if parts:  # If parts exist (like piano, violin etc.)
        notes_to_parse = parts.parts[0].recurse()
    else:
        notes_to_parse = chorale.flat.notes # Use chorale directly

    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

print("✅ Total notes collected:", len(notes))
print("🎶 Sample notes:", notes[:20])

✅ Total notes collected: 4022
🎶 Sample notes: ['G4', 'D4', 'B3', 'G2', 'G4', 'D4', 'B3', 'G3', 'E4', 'C4', 'E3', 'B3', 'D5', 'D4', 'A3', 'F#3', 'B4', 'D4', 'G3', 'G3']


In [3]:
# 🎼 STEP 3: Preprocess the Data
import numpy as np

# Create a sorted list of unique notes
unique_notes = sorted(set(notes))
n_vocab = len(unique_notes)

# Map each note to a number
note_to_int = {note: number for number, note in enumerate(unique_notes)}

# Sequence length (how many notes to consider before predicting the next one)
sequence_length = 100
network_input = []
network_output = []

# Create input/output sequences
for i in range(0, len(notes) - sequence_length):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[n] for n in sequence_in])
    network_output.append(note_to_int[sequence_out])

n_patterns = len(network_input)

# Reshape and normalize
X = np.reshape(network_input, (n_patterns, sequence_length, 1))
X = X / float(n_vocab)

# Convert outputs to one-hot encoding
from tensorflow.keras.utils import to_categorical
y = to_categorical(network_output)

print(f"✅ Total Sequences: {n_patterns}")
print(f"🧠 Input Shape: {X.shape}")


✅ Total Sequences: 3922
🧠 Input Shape: (3922, 100, 1)


In [4]:
# 🎼 STEP 4: Build the LSTM Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Activation

model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

print("✅ Model built successfully!")
model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


✅ Model built successfully!


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 100, 256)       │       264,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 256)            │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 59)             │        15,163 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 59)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 870,459 (3.32 MB)

 Trainable params: 870,459 (3.32 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# 🎵 STEP 5: Train the LSTM Model
history = model.fit(X, y, epochs=50, batch_size=64)

print("🎶 Training complete!")


Epoch 1/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 7s 34ms/step - loss: 3.7298
Epoch 2/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 4s 52ms/step - loss: 3.5489
Epoch 3/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 3.5258
Epoch 4/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 3.5149
Epoch 5/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 3.4883
Epoch 6/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 3.5038
Epoch 7/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 3.4815
Epoch 8/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 3.4726
Epoch 9/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 3.4706
Epoch 10/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 3.4497
Epoch 11/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 3.3974
Epoch 12/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 3.4190
Epoch 13/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 3.3967
Epoch 14/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.3921
Epoch 15/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.4017
Epoc

In [6]:
# 🎵 STEP 6: Generate Music Using the Trained Model
import random

# Pick a random starting point from your training data
start_index = random.randint(0, len(X) - 1)
pattern = X[start_index]
generated_notes = []

# Generate 200 new notes
for i in range(200):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction = model.predict(prediction_input, verbose=0)

    index = np.argmax(prediction)
    result = list(unique_notes)[index]
    generated_notes.append(result)

    pattern = np.append(pattern, [[index / len(unique_notes)]], axis=0)
    pattern = pattern[1:]

print("🎹 Generated Notes Sample:")
print(generated_notes[:50])  # Show first 50 notes


🎹 Generated Notes Sample:
['C4', 'E5', 'G#4', 'D4', 'B3', 'A4', 'C4', 'A3', 'E5', 'B4', 'B3', 'G#3', 'D5', 'C5', 'E4', 'C4', 'A3', 'F#4', 'D4', 'G#4', 'E4', 'E3', 'D5', 'G#4', 'E4', 'B2', 'C5', 'A4', 'E4', 'C3', 'D3', 'B4', 'G#4', 'E4', 'E3', 'D4', 'A4', 'A4', 'C4', 'F3', 'G4', 'B4', 'F4', 'D4', 'E3', 'C5', 'D3', 'C5', 'C5', 'E4']


In [7]:
# 🎵 STEP 7: Convert Generated Notes to MIDI file
from music21 import stream, note, chord

output_notes = []

for pattern in generated_notes:
    # If pattern is a chord
    if (' ' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split()
        chord_notes = [note.Note(n) for n in notes_in_chord]
        new_chord = chord.Chord(chord_notes)
        new_chord.duration.quarterLength = 0.5
        output_notes.append(new_chord)
    else:
        new_note = note.Note(pattern)
        new_note.duration.quarterLength = 0.5
        output_notes.append(new_note)

# Create a stream (music timeline)
midi_stream = stream.Stream(output_notes)

# Save to MIDI
midi_file_path = "AI_Composition.mid"
midi_stream.write("midi", fp=midi_file_path)

print("✅ MIDI file generated successfully! Download below 👇")


✅ MIDI file generated successfully! Download below 👇


In [8]:
from google.colab import files
files.download("AI_Composition.mid")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>